### Importa Bibliotecas

In [1]:
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler
import joblib
import statsmodels.api as sm

### Parâmetros

In [2]:
data_corte = '20220901' # período validação
choques = [-10, -5,0, 5, 10]

### Funções

In [3]:
def aplicar_choque_regressao_linear(df, modelo, percentual_choque):
    """
    Aplica um choque de percentual_choque nas variáveis do DataFrame e realiza projeções usando um modelo de regressão linear.
    """
    df_chocado = df.drop(columns='DATA_REFERENCIA').copy()
    df_chocado = df_chocado * (1 + percentual_choque / 100)
    
    # Adiciona a coluna de intercepto, se necessário
    df_chocado = sm.add_constant(df_chocado.drop(columns='const', errors='ignore'))

    predicoes = modelo.predict(df_chocado)
    return predicoes

In [4]:
def aplicar_choque_sarimax(df, modelo, percentual_choque):
    """
    Aplica um choque de percentual_choque nas variáveis exógenas do DataFrame e realiza projeções usando um modelo SARIMAX.
    """
    df_chocado = df.copy()
    df_chocado *= (1 + percentual_choque / 100)
    predicoes = modelo.forecast(steps=len(df_chocado), exog=df_chocado)
    return predicoes

In [5]:
def aplicar_choque_var(df, modelo, percentual_choque):
    """
    Aplica um choque de percentual_choque nas variáveis do DataFrame e realiza projeções usando um modelo VAR.
    """
    df_chocado = df.copy()
    df_chocado *= (1 + percentual_choque / 100)

    # Ajusta o modelo com os dados originais
    modelo_ajustado = modelo.fit()
    
    # Realiza a projeção com o modelo ajustado
    predicoes = modelo_ajustado.forecast(y=modelo_ajustado.endog, steps=len(df_chocado))

    # Converte as previsões em um DataFrame com as mesmas colunas do modelo
    colunas = modelo_ajustado.names  # Nomes das variáveis no modelo
    predicoes_df = pd.DataFrame(predicoes, columns=colunas)

    return predicoes_df

In [6]:
def aplicar_choque_xgboost(df, modelo, percentual_choque):
    """
    Aplica um choque de percentual_choque nas variáveis do DataFrame e realiza projeções usando um modelo XGBoost.
    """
    df_chocado = df.copy()
    
    # Normaliza as variáveis antes de aplicar o choque
    scaler = joblib.load("scaler_padrao.pkl")  # carrega o scaler ajustado
    df_normalizado = scaler.transform(df_chocado.drop(columns='DATA_REFERENCIA'))

    # Aplica o choque
    df_normalizado *= (1 + percentual_choque / 100)

    # Realiza a previsão
    predicoes = modelo.predict(df_normalizado)
    return predicoes

### Importa Modelos

In [7]:
# modelo regressão linear
with open('modelo_linear.pkl', 'rb') as arquivo:
    reg_linear = pickle.load(arquivo)
    
# modelo sarimax
with open('modelo_sarimax_completo.pkl', 'rb') as arquivo:
    sarimax = pickle.load(arquivo)

# modelo var
with open('modelo_var_completo.pkl', 'rb') as arquivo:
    var = pickle.load(arquivo)

# modelo regressão xgboost
with open('modelo_xgb_completo.pkl', 'rb') as arquivo:
    xgb = pickle.load(arquivo)

# Carrega o scaler salvo
scaler = joblib.load('scaler_padrao.pkl')

### Importa Base

In [8]:
df = pd.read_parquet('base.parquet')

In [9]:
# cria variáveis para o modelo VAR
df['DIFF_CAMBIO_LAG_1'] = df['CAMBIO_LAG_1'].diff()
df['DIFF_CAMBIO_LAG_3'] = df['CAMBIO_LAG_3'].diff()

In [10]:
df = df[df['DATA_REFERENCIA'] > data_corte]

In [11]:
df.head()

,DATA_REFERENCIA,PD,IPCA_LAG_1,IPCA_LAG_3,IPCA_LAG_6,IPCA_LAG_12,DESEMPREGO_LAG_1,DESEMPREGO_LAG_3,DESEMPREGO_LAG_6,DESEMPREGO_LAG_12,...,SELIC_LAG_1,SELIC_LAG_3,SELIC_LAG_6,SELIC_LAG_12,CAMBIO_LAG_1,CAMBIO_LAG_3,CAMBIO_LAG_6,CAMBIO_LAG_12,DIFF_CAMBIO_LAG_1,DIFF_CAMBIO_LAG_3
93,2022-10-01,3.159615,-3.425441,-7.861667,13.488658,16.074844,8.7,9.1,10.5,12.1,...,13.623245,13.084800,10.427491,6.041083,5.2370,5.3681,4.7580,5.5400,0.0937,0.3189
94,2022-11-01,3.187230,7.315329,-4.234939,5.788194,12.015841,8.3,8.9,9.8,11.6,...,12.950555,14.979655,13.084800,7.314325,5.2503,5.1433,4.9551,5.5569,0.0133,-0.2248
95,2022-12-01,3.158198,5.031984,-3.425441,8.343738,9.119872,8.1,8.7,9.3,11.1,...,12.950555,13.623245,12.950555,9.641534,5.2747,5.2370,5.0492,5.6514,0.0244,0.0937
96,2023-01-01,3.091412,7.698614,7.315329,-7.861667,6.676334,7.9,8.3,9.1,11.2,...,14.299605,12.950555,13.084800,9.120415,5.2424,5.2503,5.3681,5.5341,-0.0323,0.0133
97,2023-02-01,3.203317,6.548183,5.031984,-4.234939,12.817282,8.4,8.1,8.9,11.2,...,14.299605,12.950555,14.979655,9.511041,5.2007,5.2747,5.1433,5.1966,-0.0417,0.0244


### Separa Target

In [12]:
df_target = df[['DATA_REFERENCIA','PD']]

### Cria bases de acordo com cada modelo

In [13]:
df_reg_linear = df[['DATA_REFERENCIA','IPCA_LAG_1','IPCA_LAG_3']]

In [14]:
df_reg_linear

,DATA_REFERENCIA,IPCA_LAG_1,IPCA_LAG_3
93,2022-10-01,-3.425441,-7.861667
94,2022-11-01,7.315329,-4.234939
95,2022-12-01,5.031984,-3.425441
96,2023-01-01,7.698614,7.315329
97,2023-02-01,6.548183,5.031984
98,2023-03-01,10.558872,7.698614
99,2023-04-01,8.861143,6.548183
100,2023-05-01,7.570845,10.558872
101,2023-06-01,2.794262,8.861143
102,2023-07-01,-0.955390,7.570845


In [15]:
df_sarimax = df[['DATA_REFERENCIA','IPCA_LAG_6']]

In [16]:
df_sarimax

,DATA_REFERENCIA,IPCA_LAG_6
93,2022-10-01,13.488658
94,2022-11-01,5.788194
95,2022-12-01,8.343738
96,2023-01-01,-7.861667
97,2023-02-01,-4.234939
98,2023-03-01,-3.425441
99,2023-04-01,7.315329
100,2023-05-01,5.031984
101,2023-06-01,7.698614
102,2023-07-01,6.548183


In [17]:
df_var = df[['DATA_REFERENCIA','PD','DESEMPREGO_LAG_12','DIFF_CAMBIO_LAG_1','DIFF_CAMBIO_LAG_3']]

In [18]:
df_var

,DATA_REFERENCIA,PD,DESEMPREGO_LAG_12,DIFF_CAMBIO_LAG_1,DIFF_CAMBIO_LAG_3
93,2022-10-01,3.159615,12.1,0.0937,0.3189
94,2022-11-01,3.187230,11.6,0.0133,-0.2248
95,2022-12-01,3.158198,11.1,0.0244,0.0937
96,2023-01-01,3.091412,11.2,-0.0323,0.0133
97,2023-02-01,3.203317,11.2,-0.0417,0.0244
98,2023-03-01,3.192858,11.1,-0.0290,-0.0323
99,2023-04-01,3.218239,10.5,0.0398,-0.0417
100,2023-05-01,3.258903,9.8,-0.1918,-0.0290
101,2023-06-01,3.286762,9.3,-0.0369,0.0398
102,2023-07-01,3.364353,9.1,-0.1312,-0.1918


In [19]:
df_xgb = df.drop(columns=['PD','DIFF_CAMBIO_LAG_1','DIFF_CAMBIO_LAG_3'])

In [20]:
df_xgb

,DATA_REFERENCIA,IPCA_LAG_1,IPCA_LAG_3,IPCA_LAG_6,IPCA_LAG_12,DESEMPREGO_LAG_1,DESEMPREGO_LAG_3,DESEMPREGO_LAG_6,DESEMPREGO_LAG_12,PIB_LAG_1,...,PIB_LAG_6,PIB_LAG_12,SELIC_LAG_1,SELIC_LAG_3,SELIC_LAG_6,SELIC_LAG_12,CAMBIO_LAG_1,CAMBIO_LAG_3,CAMBIO_LAG_6,CAMBIO_LAG_12
93,2022-10-01,-3.425441,-7.861667,13.488658,16.074844,8.7,9.1,10.5,12.1,836785.8,...,819727.6,755323.0,13.623245,13.084800,10.427491,6.041083,5.2370,5.3681,4.7580,5.5400
94,2022-11-01,7.315329,-4.234939,5.788194,12.015841,8.3,8.9,9.8,11.6,877947.5,...,826052.1,771558.9,12.950555,14.979655,13.084800,7.314325,5.2503,5.1433,4.9551,5.5569
95,2022-12-01,5.031984,-3.425441,8.343738,9.119872,8.1,8.7,9.3,11.1,879514.9,...,826025.7,782833.5,12.950555,13.623245,12.950555,9.641534,5.2747,5.2370,5.0492,5.6514
96,2023-01-01,7.698614,7.315329,-7.861667,6.676334,7.9,8.3,9.1,11.2,883886.2,...,852909.9,722909.9,14.299605,12.950555,13.084800,9.120415,5.2424,5.2503,5.3681,5.5341
97,2023-02-01,6.548183,5.031984,-4.234939,12.817282,8.4,8.1,8.9,11.2,812834.6,...,854004.1,754147.4,14.299605,12.950555,14.979655,9.511041,5.2007,5.2747,5.1433,5.1966
98,2023-03-01,10.558872,7.698614,-3.425441,21.269854,8.6,7.9,8.7,11.1,831224.5,...,836785.8,838868.0,11.616115,14.299605,13.623245,11.748906,5.1717,5.2424,5.2370,4.9684
99,2023-04-01,8.861143,6.548183,7.315329,13.488658,8.8,8.4,8.3,10.5,938679.8,...,877947.5,819727.6,14.979655,14.299605,12.950555,10.427491,5.2115,5.2007,5.2503,4.7580
100,2023-05-01,7.570845,10.558872,5.031984,5.788194,8.5,8.6,8.1,9.8,904664.8,...,879514.9,826052.1,11.616115,11.616115,12.950555,13.084800,5.0197,5.1717,5.2747,4.9551
101,2023-06-01,2.794262,8.861143,7.698614,8.343738,8.3,8.8,7.9,9.3,903238.1,...,883886.2,826025.7,14.299605,14.979655,14.299605,12.950555,4.9828,5.2115,5.2424,5.0492
102,2023-07-01,-0.955390,7.570845,6.548183,-7.861667,8.0,8.5,8.4,9.1,893074.9,...,812834.6,852909.9,13.623245,11.616115,14.299605,13.084800,4.8516,5.0197,5.2007,5.3681


### Impactos

***Regressão Linear***

In [21]:
df_linear_impacto = pd.DataFrame()

for choque in choques:
    df_linear_impacto[f"CHOQUE_{choque}%"] = aplicar_choque_regressao_linear(df_reg_linear, reg_linear, choque)

df_linear_impacto = pd.concat([df_target, df_linear_impacto], axis=1)

In [22]:
reg_linear.params

const         3.593441
IPCA_LAG_1   -0.013746
IPCA_LAG_3   -0.013371
dtype: float64

In [23]:
df_linear_impacto

,DATA_REFERENCIA,PD,CHOQUE_-10%,CHOQUE_-5%,CHOQUE_0%,CHOQUE_5%,CHOQUE_10%
93,2022-10-01,3.159615,3.730424,3.738034,3.745644,3.753254,3.760864
94,2022-11-01,3.187230,3.553900,3.551703,3.549506,3.547310,3.545113
95,2022-12-01,3.158198,3.572407,3.571239,3.570070,3.568902,3.567733
96,2023-01-01,3.091412,3.410167,3.399985,3.389803,3.379621,3.369439
97,2023-02-01,3.203317,3.451876,3.444011,3.436147,3.428282,3.420417
98,2023-03-01,3.192858,3.370168,3.357764,3.345360,3.332956,3.320552
99,2023-04-01,3.218239,3.405016,3.394548,3.384080,3.373611,3.363143
100,2023-05-01,3.258903,3.372716,3.360453,3.348191,3.335928,3.323666
101,2023-06-01,3.286762,3.452240,3.444395,3.436551,3.428706,3.420862
102,2023-07-01,3.364353,3.514156,3.509752,3.505347,3.500942,3.496537


***SARIMAX***

In [24]:
df_sarimax_impacto = pd.DataFrame()

for choque in choques:
    df_sarimax_impacto[f"CHOQUE_{choque}%"] = aplicar_choque_sarimax(df_sarimax.drop(columns='DATA_REFERENCIA'), sarimax, choque)

In [25]:
# Garantir que os índices estejam alinhados
df_sarimax_impacto.index = df_target['DATA_REFERENCIA']

# Concatenar os DataFrames lado a lado
df_sarimax_impacto = pd.concat([df_target.set_index('DATA_REFERENCIA'), df_sarimax_impacto], axis=1)

# Resetar o índice, se necessário
df_sarimax_impacto.reset_index(inplace=True)

df_sarimax_impacto

,DATA_REFERENCIA,PD,CHOQUE_-10%,CHOQUE_-5%,CHOQUE_0%,CHOQUE_5%,CHOQUE_10%
0,2022-10-01,3.159615,3.278807,3.276575,3.274342,3.272110,3.269878
1,2022-11-01,3.187230,3.233311,3.232353,3.231395,3.230437,3.229479
2,2022-12-01,3.158198,3.205027,3.203646,3.202265,3.200884,3.199503
3,2023-01-01,3.091412,3.254020,3.255321,3.256622,3.257923,3.259224
4,2023-02-01,3.203317,3.317878,3.318578,3.319279,3.319980,3.320681
5,2023-03-01,3.192858,3.312086,3.312653,3.313220,3.313787,3.314354
6,2023-04-01,3.218239,3.221650,3.220439,3.219228,3.218018,3.216807
7,2023-05-01,3.258903,3.253136,3.252303,3.251470,3.250637,3.249804
8,2023-06-01,3.286762,3.257296,3.256022,3.254748,3.253474,3.252200
9,2023-07-01,3.364353,3.270801,3.269717,3.268633,3.267550,3.266466


***VAR***

In [26]:
df_var_impacto = pd.DataFrame()

for choque in choques:
    df_var_impacto[f"CHOQUE_{choque}%"] = aplicar_choque_var(df_var.drop(columns='DATA_REFERENCIA'), var, choque)['PD']

In [27]:
# Garantir que os índices estejam alinhados
df_var_impacto.index = df_target['DATA_REFERENCIA']

# Concatenar os DataFrames lado a lado
df_var_impacto = pd.concat([df_target.set_index('DATA_REFERENCIA'), df_var_impacto], axis=1)

# Resetar o índice, se necessário
df_var_impacto.reset_index(inplace=True)

df_var_impacto

,DATA_REFERENCIA,PD,CHOQUE_-10%,CHOQUE_-5%,CHOQUE_0%,CHOQUE_5%,CHOQUE_10%
0,2022-10-01,3.159615,3.143712,3.143712,3.143712,3.143712,3.143712
1,2022-11-01,3.187230,3.169033,3.169033,3.169033,3.169033,3.169033
2,2022-12-01,3.158198,3.199649,3.199649,3.199649,3.199649,3.199649
3,2023-01-01,3.091412,3.227319,3.227319,3.227319,3.227319,3.227319
4,2023-02-01,3.203317,3.251060,3.251060,3.251060,3.251060,3.251060
5,2023-03-01,3.192858,3.271380,3.271380,3.271380,3.271380,3.271380
6,2023-04-01,3.218239,3.288780,3.288780,3.288780,3.288780,3.288780
7,2023-05-01,3.258903,3.303644,3.303644,3.303644,3.303644,3.303644
8,2023-06-01,3.286762,3.316293,3.316293,3.316293,3.316293,3.316293
9,2023-07-01,3.364353,3.327010,3.327010,3.327010,3.327010,3.327010


***XgBoost***

In [28]:
df_xgb_impacto = pd.DataFrame()

for choque in choques:
    df_xgb_impacto[f"CHOQUE_{choque}%"] = aplicar_choque_xgboost(df_xgb, xgb, choque)

In [29]:
# Garantir que os índices estejam alinhados
df_xgb_impacto.index = df_target['DATA_REFERENCIA']

# Concatenar os DataFrames lado a lado
df_xgb_impacto = pd.concat([df_target.set_index('DATA_REFERENCIA'), df_xgb_impacto], axis=1)

# Resetar o índice, se necessário
df_xgb_impacto.reset_index(inplace=True)

df_xgb_impacto

,DATA_REFERENCIA,PD,CHOQUE_-10%,CHOQUE_-5%,CHOQUE_0%,CHOQUE_5%,CHOQUE_10%
0,2022-10-01,3.159615,3.313320,3.293666,3.293666,3.286564,3.286564
1,2022-11-01,3.187230,3.296648,3.296648,3.296648,3.292651,3.292651
2,2022-12-01,3.158198,3.292651,3.292651,3.292651,3.292651,3.292651
3,2023-01-01,3.091412,3.292195,3.286414,3.286414,3.286414,3.286414
4,2023-02-01,3.203317,3.302803,3.298690,3.303638,3.303638,3.293516
5,2023-03-01,3.192858,3.329742,3.329742,3.320455,3.313353,3.313353
6,2023-04-01,3.218239,3.292651,3.292651,3.292651,3.292651,3.292651
7,2023-05-01,3.258903,3.346248,3.346248,3.346248,3.346248,3.346248
8,2023-06-01,3.286762,3.323703,3.323703,3.323703,3.323703,3.323703
9,2023-07-01,3.364353,3.323703,3.323703,3.323703,3.323703,3.323703


### Variação Média

***Linear***

In [30]:
# Criar um DataFrame para armazenar as variações médias
variacoes_medias = {}

# Iterar pelas colunas de choque
for coluna in df_linear_impacto.columns:
    if 'CHOQUE' in coluna:  # Apenas colunas de choque
        # Calcular a variação percentual média
        variacao_media = ((df_linear_impacto[coluna] - df_linear_impacto['CHOQUE_0%']) / df_linear_impacto['CHOQUE_0%']).mean() * 100
        variacoes_medias[coluna] = variacao_media

# Transformar o dicionário em um DataFrame para exibição
df_variacoes = pd.DataFrame(list(variacoes_medias.items()), columns=['CHOQUE', 'Variacao_Media_%'])

# Exibir o DataFrame
df_variacoes

,CHOQUE,Variacao_Media_%
0,CHOQUE_-10%,0.301939
1,CHOQUE_-5%,0.150969
2,CHOQUE_0%,0.000000
3,CHOQUE_5%,-0.150969
4,CHOQUE_10%,-0.301939


***SARIMAX***

In [31]:
# Criar um DataFrame para armazenar as variações médias
variacoes_medias = {}

# Iterar pelas colunas de choque
for coluna in df_sarimax_impacto.columns:
    if 'CHOQUE' in coluna:  # Apenas colunas de choque
        # Calcular a variação percentual média
        variacao_media = ((df_sarimax_impacto[coluna] - df_sarimax_impacto['CHOQUE_0%']) / df_sarimax_impacto['CHOQUE_0%']).mean() * 100
        variacoes_medias[coluna] = variacao_media

# Transformar o dicionário em um DataFrame para exibição
df_variacoes = pd.DataFrame(list(variacoes_medias.items()), columns=['CHOQUE', 'Variacao_Media_%'])

# Exibir o DataFrame
df_variacoes

,CHOQUE,Variacao_Media_%
0,CHOQUE_-10%,0.046167
1,CHOQUE_-5%,0.023084
2,CHOQUE_0%,0.000000
3,CHOQUE_5%,-0.023084
4,CHOQUE_10%,-0.046167


***VAR***

In [32]:
# Criar um DataFrame para armazenar as variações médias
variacoes_medias = {}

# Iterar pelas colunas de choque
for coluna in df_var_impacto.columns:
    if 'CHOQUE' in coluna:  # Apenas colunas de choque
        # Calcular a variação percentual média
        variacao_media = ((df_var_impacto[coluna] - df_var_impacto['CHOQUE_0%']) / df_var_impacto['CHOQUE_0%']).mean() * 100
        variacoes_medias[coluna] = variacao_media

# Transformar o dicionário em um DataFrame para exibição
df_variacoes = pd.DataFrame(list(variacoes_medias.items()), columns=['CHOQUE', 'Variacao_Media_%'])

# Exibir o DataFrame
df_variacoes

,CHOQUE,Variacao_Media_%
0,CHOQUE_-10%,0.0
1,CHOQUE_-5%,0.0
2,CHOQUE_0%,0.0
3,CHOQUE_5%,0.0
4,CHOQUE_10%,0.0


***XgBoost***

In [33]:
# Criar um DataFrame para armazenar as variações médias
variacoes_medias = {}

# Iterar pelas colunas de choque
for coluna in df_xgb_impacto.columns:
    if 'CHOQUE' in coluna:  # Apenas colunas de choque
        # Calcular a variação percentual média
        variacao_media = ((df_xgb_impacto[coluna] - df_xgb_impacto['CHOQUE_0%']) / df_xgb_impacto['CHOQUE_0%']).mean() * 100
        variacoes_medias[coluna] = variacao_media

# Transformar o dicionário em um DataFrame para exibição
df_variacoes = pd.DataFrame(list(variacoes_medias.items()), columns=['CHOQUE', 'Variacao_Media_%'])

# Exibir o DataFrame
df_variacoes

,CHOQUE,Variacao_Media_%
0,CHOQUE_-10%,0.084506
1,CHOQUE_-5%,0.016680
2,CHOQUE_0%,0.000000
3,CHOQUE_5%,-0.036715
4,CHOQUE_10%,-0.094776
